<a href="https://colab.research.google.com/github/erdalbucan/Colab/blob/main/Makine_%C3%96%C4%9Frenmesi_Hata_Metrikleri_mse_%2C_rmse_%2C_mae_%2C_mape_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#https://www.kaggle.com/datasets/mirichoi0218/insurance insurance.cvs yi indiriyoruz

In [71]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [16]:
df = pd.read_csv('insurance.csv')
df.head(3)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620


In [17]:
# daatamizda iki tur veri var. numerik ve kategorisel.
#kategorusel verimizi dummies ediyoruz
df = pd.get_dummies(df, columns= ['sex','smoker','region'], drop_first=True)
#veimiz tumuyle numerik oldu, modellemeye hazir.
df.head(3)

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.90,0,16884.9240,False,True,False,False,True
1,18,33.77,1,1725.5523,True,False,False,True,False
2,28,33.00,3,4449.4620,True,False,False,True,False


In [20]:
#modellemeye baslaayiruz, oncelikle Y ve X leri belirleyelim y = charges
y = df[['charges']]
x = df.drop(df[['charges']],axis=1)
x.head(3)

,age,bmi,children,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.90,0,False,True,False,False,True
1,18,33.77,1,True,False,False,True,False
2,28,33.00,3,True,False,False,True,False


In [21]:
# modeli olusturalim
lm = LinearRegression()
model = lm.fit(x,y) #modelimize ogretiyoruz

In [22]:
# scorumuzu ohrenelim
model.score(x,y)
# scorumuz 0.7509130345985207 imis %75 doruluk tahmini

0.7509130345985207

In [45]:
model.predict([[45,34,3,1,0,0,0,1]])
#45 yas, (bmi34 ???), 3 cocuklu, erkek, SIGARA ICMEYEN, southwest te yasiyor
#yillik masraf 11487

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[11487.71099553]])

In [48]:
model.predict([[45,34,3,1,1,0,0,1]])
#45 yas, (bmi34 ???), 3 cocuklu, erkek, SIGARA ICIYOR, southwest te yasiyor
#yillik masraf 35336
# sigara icen ile icmeyen arasindaki yillik harcama farki 23849 eur )))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[35336.24553745]])

In [51]:
#hatalari ogrenelim
df_hata = pd.DataFrame() # bos bir data frime olusturduk
df_hata['y'] = y # yukardaki gercek sonuclari (y) bu data frimeye copyaladik
df_hata.head(3)

,y
0,16884.9240
1,1725.5523
2,4449.4620


In [54]:
# simdi tahminleri sutun yapacagiz ve tahmin ile gercekveri(y) arasindaki farklaari inceleyecegiz
y_tahmin = model.predict(x) # eldeki x lerle buyun tahminleri yap
df_hata[['tahmin']] = y_tahmin
df_hata.head(3)
# Y=olmasi gereken gercek deger. tahmin = modelin tahmin ettigi deger
# mesela 0.satirdaa 16884 tahmin etmesi gerekirken 25293 tahmin etmis ciddi fark var

,y,tahmin
0,16884.9240,25293.713028
1,1725.5523,3448.602834
2,4449.4620,6706.988491


In [56]:
# tahmindeki fark yani error lari bulalim
df_hata['error'] = y -(y_tahmin)
df_hata.head(3)
# hatalar - veya + olarak geldi. - leri mutlak deger yani + yapmamiz gerekir yoksa birbirini noturlediginden uzaklik bulmak zorlasir.

,y,tahmin,error
0,16884.9240,25293.713028,-8408.789028
1,1725.5523,3448.602834,-1723.050534
2,4449.4620,6706.988491,-2257.526491


In [60]:
# errorlarain karelerini alirsak butun veriler pozitife goder ortalamayi alabilriz
df_hata['squared_error'] = df_hata['error']**2
df_hata.head(5)
#butun squared_error hatalaar pozitif oldu
#MSE MSE MSE

,y,tahmin,error,squared_error
0,16884.92400,25293.713028,-8408.789028,7.070773e+07
1,1725.55230,3448.602834,-1723.050534,2.968903e+06
2,4449.46200,6706.988491,-2257.526491,5.096426e+06
3,21984.47061,3754.830163,18229.640447,3.323198e+08
4,3866.85520,5592.493386,-1725.638186,2.977827e+06


In [62]:
# bir diger hatalari pozitive cevirme yolu mutlak degerler yani - leri + yapmak
df_hata['abs_error'] = np.abs(df_hata['error'])
df_hata.head(5)
#error ve abs_error birbirinin ayni rakamlar sadece - ler + oldu
#MAE MAE

,y,tahmin,error,squared_error,abs_error
0,16884.92400,25293.713028,-8408.789028,7.070773e+07,8408.789028
1,1725.55230,3448.602834,-1723.050534,2.968903e+06,1723.050534
2,4449.46200,6706.988491,-2257.526491,5.096426e+06,2257.526491
3,21984.47061,3754.830163,18229.640447,3.323198e+08,18229.640447
4,3866.85520,5592.493386,-1725.638186,2.977827e+06,1725.638186


In [67]:
df_hata['percent_error'] = np.abs((y-y_tahmin)/y) #hata yuzdeleri
df_hata.head(5)
#MAPE MAPE

,y,tahmin,error,squared_error,abs_error,percent_error
0,16884.92400,25293.713028,-8408.789028,7.070773e+07,8408.789028,0.498006
1,1725.55230,3448.602834,-1723.050534,2.968903e+06,1723.050534,0.998550
2,4449.46200,6706.988491,-2257.526491,5.096426e+06,2257.526491,0.507371
3,21984.47061,3754.830163,18229.640447,3.323198e+08,18229.640447,0.829205
4,3866.85520,5592.493386,-1725.638186,2.977827e+06,1725.638186,0.446264


In [70]:
# hedafimin meaaanlari bulmaak yani ortaalamalari
df_hata.mean()
#y                1.327042e+04
# tahmin           1.327042e+04
# error            2.289371e-12
# squared_error    3.650189e+07
# abs_error        4.170887e+03
# percent_error    4.203527e-01
# dtype: float64

# MEAN LAR

y                1.327042e+04
tahmin           1.327042e+04
error            2.289371e-12
squared_error    3.650189e+07
abs_error        4.170887e+03
percent_error    4.203527e-01
dtype: float64

In [72]:
# YUKARDA YAPTIGIMIZ ISLEMLERIN PTATIK VE KOLAY KISMI
#from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [74]:
#MSE
mean_squared_error(y, y_tahmin) # suared_error
#36501893.00741544

36501893.00741544

In [76]:
#MAE
mean_absolute_error(y, y_tahmin) #abs_error
#4170.886894163586

4170.886894163586

In [77]:
#MAPE
mean_absolute_percentage_error(y, y_tahmin) #percent_error    4.203527e-01
#0.4203526847372702

0.4203526847372702